In [1]:
import numpy as np
import pandas as pd
from load_data import load_train_data
from lda import train_lda, predict
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load data
feat,params,daq = load_train_data('train_data_raw_AB.mat')

In [3]:
# Build NN model
model = Sequential([
    Dense(units=60, input_shape=(60,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=7, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [32]:
# Index data
# grp = 1 (static), 2 (dynamic), 3 (0g), 4 (400g), 5 (500g), 6 (600g)
sub = 15
grp = 3
ind = (params[:,0] < sub) & (params[:,3] == grp)

# Shuffle and split data
feat_s, label_s = shuffle(feat[ind,:],params[ind,-2,np.newaxis])
X_train, X_test, y_train, y_test = train_test_split(feat_s, label_s, test_size=0.3, random_state=0, stratify=label_s)

# Reshape and scale data
y_train = np.squeeze(y_train-1)
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_train)
X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)

In [33]:
# Fit model
model.fit(x=X_train_n, y=y_train, validation_split=0.1, batch_size=32, epochs=30, verbose=2)

Epoch 1/30
965/965 - 1s - loss: 1.1077 - accuracy: 0.6561 - val_loss: 0.8965 - val_accuracy: 0.6950
Epoch 2/30
965/965 - 1s - loss: 0.8095 - accuracy: 0.7223 - val_loss: 0.7701 - val_accuracy: 0.7397
Epoch 3/30
965/965 - 1s - loss: 0.7223 - accuracy: 0.7570 - val_loss: 0.7099 - val_accuracy: 0.7612
Epoch 4/30
965/965 - 1s - loss: 0.6753 - accuracy: 0.7757 - val_loss: 0.6764 - val_accuracy: 0.7784
Epoch 5/30
965/965 - 1s - loss: 0.6441 - accuracy: 0.7873 - val_loss: 0.6502 - val_accuracy: 0.7866
Epoch 6/30
965/965 - 1s - loss: 0.6193 - accuracy: 0.7972 - val_loss: 0.6280 - val_accuracy: 0.7956
Epoch 7/30
965/965 - 1s - loss: 0.5991 - accuracy: 0.8033 - val_loss: 0.6097 - val_accuracy: 0.8006
Epoch 8/30
965/965 - 1s - loss: 0.5819 - accuracy: 0.8090 - val_loss: 0.5942 - val_accuracy: 0.8035
Epoch 9/30
965/965 - 1s - loss: 0.5667 - accuracy: 0.8132 - val_loss: 0.5809 - val_accuracy: 0.8087
Epoch 10/30
965/965 - 1s - loss: 0.5534 - accuracy: 0.8174 - val_loss: 0.5695 - val_accuracy: 0.8120

In [34]:
# Evaluate test set
out = np.argmax(model.predict(x=X_test_n), axis = 1)
np.sum(out.reshape(y_test.shape) == y_test - 1)/y_test.shape[0]

0.8531292517006803

In [54]:
sub = 14
grp = 3
ind = (params[:,0] == sub) & (params[:,3] != grp)

# Shuffle and split data
X_test_new, y_test_new = shuffle(feat[ind,:],params[ind,-2,np.newaxis])
X_test_new = scaler.transform(X_test_new)
out = np.argmax(model.predict(x=X_test_new), axis = 1)
np.sum(out.reshape(y_test_new.shape) == y_test_new - 1)/y_test_new.shape[0]

0.7724

In [121]:
w,c = train_lda(X_train_n,y_train)
X_test_n = scaler.transform(X_test)
out = predict(X_test_n,w,c)
np.sum(out.reshape(y_test.shape) == y_test-1)/y_test.shape[0]

0.9933333333333333